# Word Embeddings - Word2Vec and GloVe

- Let us build our first word embeddings ...
- **Important**: for all of the tasks below please make use of the provided methods given by the `gensim` and `nltk` Python module
- This is a  step-by-step coding example how to get from word representation to the corresponding word embeddings in order to use them for various downstream ML/DL-driven learning algroithms
- Have look at the `GENSIM` and `NLTK-LM` documentation: https://radimrehurek.com/gensim/ and https://www.nltk.org/api/nltk.lm.html

## Load Pre-Trained Word2Vec and GloVe Embeddings

- There exist a portfolio of existing word embeddings, trained on the basis of different text corpora and various underlying algorithmic concepts, such as `Word2Vec` or `GloVe`. Look up the `Python` method, which reports all the different models and training corpora which are available for download (Hint: check the dictonary `gensim.downloader.info()` for `models`) 
- Use the `api` module to `load` the pre-trained `Word2Vec` and `GloVe` model, trained on `google-news-300` and `twitter-200` (in case you face a Jupyter-internal `Warning: IOPub message rate exceeded`, you can either ignore it or get around it via starting Jupyter with the option `--NotebookApp.iopub_data_rate_limit=1.0e10`. No worries, the download itself, but also loading each of the two models, will take some time
- You will use both pre-loaded `Word2Vec` and `GloVe` models (for comparison), but also your own trained `Word2Vec` model

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim.downloader
import gensim.downloader as api

from matplotlib import cm
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

In [3]:
google300 = api.load("word2vec-google-news-300")
twitter200 = api.load("glove-twitter-200")

## Loading, Extracting, Preprocessing, and Tokenizing (Sentences) of the Data Corpus for Downstream Model Training

- Use the same text document (`Donald-Trump-Tweets.csv`) as in the `N-GRAM` exercise, providing a series of different Tweets from Donald Trump 
- Prepare the entire data corpus including various Trump-Tweets, using a similar procedure as within the `N-GRAM` exercise, in terms of `loading, extracting, preprocessing, and tokenizing (sentences)` the data material
    - (1) `Pandas` Python library to read the `CSV`-file, filter the column which contains the text information (`Tweet_Text`), and check for empty/invalid elements
    - (2)  Clean and prepare (e.g. `regex`, `nltk` functionalities, etc.) the entire text information, to ensure a robust sentence tokenization (categorize (`NER`) patterns like hyperlinks, elements addressed via hashtags, dates, timestamps, multiple whitespaces, upper/lower case, etc.)
    - (3) Take the entire text information as one large `training` corpus (all the 7,375 lines) for downstream training of your word embedding model (later in the Jupyter notebook)
- `Sentence tokenization:` consider the fact that a single tweet often consists of multiple sentences, which has to be taken into account while building the sentences (`num_of_tweets << num_of_sentences`). Sentences should be stored (as for the `NGRAM` exercise) in a `nested list`, e.g. `[["my", "first", "sentences"], ["my", "second", "sentence"]]` (each sentence is represented as list of words (word vector $\vec{w}$))
- In terms of word embeddings, `stop words` often lead to undesired phenomena, due to a severe word-specific class imbalance. A `stop word` is very often `context word c_i` word and consequently often surrounded by topic-wise totally different words, which makes it hard to learn and derive a suitable vector representation. Therefor, in order to enhance the quality of the learned word embeddings, remove the `stop words` from the sentence list (make use of `from nltk.corpus import stopwords`, but if needed first download the pool of `stop words` via `nltk.download("stopwords")`).
- The identification of sentence beginning/end requires punctuation marks. However, when it comes to building the word embeddings, punctuation marks or any kind of remaining non-alphanumeric patterns (e.g. %, &, $, etc.) should be removed. Since you have your sentences already at this stage this is not a problem!
- How about padding the individual sentences with the start (`<s>`) and sentence end (`</s>`) tokens (as for `N-GRAM`)? Useful?

In [4]:
import re
import nltk
from nltk.lm.preprocessing import pad_both_ends
from nltk.corpus import stopwords
data = pd.read_csv('../Vorlesung3/Donald-Trump-Tweets.csv').filter(['Tweet_Text']).dropna().drop_duplicates()

text = data.values.tolist()

for i in range(len(text)):
    text[i] = re.sub('([#][\w_-]+)', 'Hashtag', str(text[i]))
    text[i] = re.sub('([@][\w_-]+)', 'Mention', str(text[i]))
    text[i] = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?\S', 'Link', str(text[i]))
    
allTweets = " ".join(text)
allTweets = re.sub("(\[\\'\.|\[\\'\"|\[\\')", '', allTweets)
allTweets = re.sub("(\\'])", '', allTweets)


nltk.download('punkt', download_dir='../NLTK_Data')
nltk.download('stopwords', download_dir='../NLTK_Data')
nltk.data.path.append('../NLTK_Data')

sentences = nltk.tokenize.sent_tokenize(allTweets.lower())
print('Anzahl Sätze aus Tweets: {}'.format(sentences.__len__()))

#remove punctuation
for i in range(len(sentences)):
    sentences[i] = re.sub('(!|\.|,|-|\&amp;|\?|\]|\\+n|:|%|\$|&|;|")', '', str(sentences[i]))
    
#Add BoS and EoS tags
sentence_word = []
stop_words = set(stopwords.words('english'))
for i in range(len(sentences)):
    words = nltk.tokenize.word_tokenize(sentences[i])
    filtered_words = [w for w in words if not w.lower() in stop_words]
    padded = list(pad_both_ends(filtered_words, n =3))
    sentence_word.append([" ".join(padded), padded])

[nltk_data] Downloading package punkt to ../NLTK_Data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to ../NLTK_Data...
[nltk_data]   Package stopwords is already up-to-date!


Anzahl Sätze aus Tweets: 10283


In [5]:
total_words = 0
for i in range(len(sentence_word)):
    total_words += (sentence_word[i][1].__len__() - 4) #substract BoS EoS token

print("Durchschnittliche Anzahl Wörter in einem Satz (ohne stopwords): {}".format(total_words/sentences.__len__()))

Durchschnittliche Anzahl Wörter in einem Satz (ohne stopwords): 7.497811922590683


## Apply Pre-Trained Word2Vec and GloVe models

- Use the pre-loaded `Word2Vec` and `GloVe` model and report the shape of a single word embedding vector for both of the models, as well as the overall size of the vocabulary. Are there differences in the vector embedding and vocabulary size and if so, how large is the difference? 
- Use both models throughout the exercise (same functionalities) in order to compute the cosine similarity $cos(\theta)$, by applying the `similarity` function for the following words pairs (you can also try others, of course):
    - `Pikachu` vs. `Pokemon`
    - `Soccer` vs. `Football`
    - `Messi` vs. `Barcelona`
    - `Pele` vs. `Ronaldo`
    - `Orca` vs. `Whale`
    - `Prof.` vs. `Dr.`
- Report the `N=10` context words which present the highest similarity in terms of a given target word (descending order). Do not implement your own function, but rather make use of an existing function, provided by the model. Use the following target words: `student, trump, boxing, hospital, matrix & python` (you can also try others, of course)
- Look up and use the `doesnt_match` function - what is the underlying purpose?
- Compute various `analogies` via `positive`and `negative` word pairs (e.g. `positive=[woman king]`, `negative=[man]`, leading to `queen`) - use the same function as for computing the most similar context words (Hint: look for an existing function of the model!)
- Get the `dense vector` (word vector) from the trained word embedding matrix via `get_vector`, as well as the `index`for a specific word via `get_index`, both for the word `europe`
- Define a function `get_word_embeddings(words)` which `returns word clouds/clusters` for each given word in a `wordlist`, including the `10` closest (highest similarity) `context word embeddings` around a given list of `target words`. The dictionary output named `word cloud` has the following structure: `target word : {context_word_1 : ndarray_embed_vec, (...) , context_word_N : ndarray_embed_vec, target_word : ndarray_embed_vec}`

```
def get_word_embeddings(model, wordlist, topn):
    w_emb = []
    w_cloud = {}
    for t_w in words:
        #TODO - use "most_similar" function with target word "t_w"
        #TODO - get embedding for context word "c_w"
        #TODO - store everything (t_w, list of c_w embeddings) in a dictionary w_cloud (key = t_w, value = list of c_w)
    return w_cloud 
````

In [8]:
print("Größe von word2vec-google-news-300: {}; Größe von glove-twitter-200: {}".format(google300.vector_size, twitter200.vector_size))

Größe von word2vec-google-news-300: 300; Größe von glove-twitter-200: 200


In [13]:
test_words=[
    ['Pikachu', 'Pokemon'],
    ['Soccer','Football'],
    ['Messi','Barcelona'],
    ['Pele','Ronaldo'],
    ['Orca','Whale'],
    ['Prof.','Dr.'],
]

for test_word in test_words:    
    print('{} und {} sind sich {} ähnlich'.format(test_word[0], test_word[1],google300.similarity(test_word[0], test_word[1])))
    
print('GloVe')
for test_word in test_words:   
    try:
        print('{} und {} sind sich {} ähnlich'.format(test_word[0], test_word[1],twitter200.similarity(test_word[0], test_word[1])))
    except:
        print("{} ist nicht im Datensatz enthalten".format(test_word[0]))

Pikachu und Pokemon sind sich 0.6372727751731873 ähnlich
Soccer und Football sind sich 0.7229189872741699 ähnlich
Messi und Barcelona sind sich 0.6065428256988525 ähnlich
Pele und Ronaldo sind sich 0.5526662468910217 ähnlich
Orca und Whale sind sich 0.5591481924057007 ähnlich
Prof. und Dr. sind sich 0.7368013262748718 ähnlich
GloVe
Pikachu ist nicht im Datensatz enthalten
Soccer ist nicht im Datensatz enthalten
Messi ist nicht im Datensatz enthalten
Pele ist nicht im Datensatz enthalten
Orca ist nicht im Datensatz enthalten
Prof. ist nicht im Datensatz enthalten


In [31]:
target_words=['student','trump','boxing','hospital','matrix','python']

for target_word in target_words:
    topn = google300.most_similar(target_word, topn=10)
    print('10 ähnlichsten Worte zu {} sind: {}'.format(target_word, [item[0] for item in topn]))

10 ähnlichsten Worte zu student sind: ['students', 'Student', 'teacher', 'stu_dent', 'faculty', 'school', 'undergraduate', 'university', 'undergraduates', 'semester']
10 ähnlichsten Worte zu trump sind: ['trumps', 'trumping', 'supersede', 'trumped', 'supercede', 'prevail', 'outweigh', 'trample', 'overshadow', 'dictate']
10 ähnlichsten Worte zu boxing sind: ['amateur_boxing', 'boxers', 'Boxing', 'boxer', 'mixed_martial_arts', 'prizefighting', 'light_heavyweight', 'welterweight', 'MMA', 'middleweight']
10 ähnlichsten Worte zu hospital sind: ['Hospital', 'hopsital', 'hosptial', 'hospitals', 'intensive_care', 'Hosptial', 'Hopsital', 'hopital', 'Intensive_Care_Unit', 'Medical_Center']
10 ähnlichsten Worte zu matrix sind: ['intramolecular', 'Si_substrate', 'covalent_bond', 'matrices', 'antiparallel', 'chromatograms', 'antibody_antigen', 'gaussian', 'matrixes', 'polyhedral']
10 ähnlichsten Worte zu python sind: ['pythons', 'Burmese_python', 'snake', 'crocodile', 'boa_constrictor', 'alligator'

In [44]:
google300.doesnt_match(['Dark Magician', 'Exodia', 'Blue-Eyes white dragon', 'Pikachu'])

'Pikachu'

In [58]:
google300.most_similar(positive=['king'])

[('kings', 0.7138045430183411),
 ('queen', 0.6510956883430481),
 ('monarch', 0.6413194537162781),
 ('crown_prince', 0.6204220056533813),
 ('prince', 0.6159993410110474),
 ('sultan', 0.5864824056625366),
 ('ruler', 0.5797567367553711),
 ('princes', 0.5646552443504333),
 ('Prince_Paras', 0.5432944297790527),
 ('throne', 0.5422105193138123)]

In [60]:
google300.get_vector('europe')

array([-1.54296875e-01, -8.00781250e-02,  1.13281250e-01,  4.70703125e-01,
       -4.15039062e-02,  4.56542969e-02, -2.03125000e-01, -3.57421875e-01,
       -5.90820312e-02,  3.12500000e-02, -1.74804688e-01, -1.82617188e-01,
       -4.19921875e-02, -2.61718750e-01,  7.47070312e-02,  2.27539062e-01,
        1.42578125e-01,  3.37890625e-01, -1.39160156e-02,  1.47705078e-02,
        1.71875000e-01, -1.94335938e-01,  2.50000000e-01,  7.23266602e-03,
       -1.15722656e-01,  4.82421875e-01, -3.78906250e-01,  1.52343750e-01,
       -1.80664062e-01,  1.41601562e-01,  1.55639648e-02,  3.32031250e-01,
       -1.04980469e-01, -1.32812500e-01, -1.25976562e-01,  4.66308594e-02,
       -3.16406250e-01, -4.24804688e-02,  3.55468750e-01,  1.95312500e-01,
       -6.73828125e-02,  6.44531250e-02,  1.33789062e-01,  3.03955078e-02,
        2.31445312e-01,  7.32421875e-02,  5.76171875e-02, -2.40478516e-02,
       -3.16406250e-01,  5.41992188e-02, -1.49414062e-01, -4.95605469e-02,
        1.53320312e-01,  

In [61]:
google300.get_index('europe')

122664

## Compute Topic-Related Word Clouds

- Apply the function `get_word_embeddings(model, wordlist, topn)` to a number of topic-specific words `topics = yoda pizza orca hamburg mojito mercedes`, by converting `topics` to a `wordlist`. In addition, please select `topn=10`, resulting in the top-10 closest neighbors (words with the highest similarity) for each of those target words. As a result, the function `get_word_embeddings(model, wordlist, topn)` returns the word cloud object (10 surrounding word embeddings + 1 target word embedding for each topic). In case $topn=0$ only the target word (no neighboring words) should be reported!
- Compute the topic-related `word cloud` information for both models: `Word2Vec` and `GloVe`
- The `word cloud` and its underlying high-dimensional word embeddings can now be used to visualize compact word relations 

In [ ]:
google300.

## Visualization Word2Vec/GloVe embeddings
- Execute the code-snippet below, by calling the function: `tsne_plot(word_cloud, topn)` which takes the previous calculated `word cloud` information of the `topics`. The method reduces the dimensionality of the exisitng $M \times N$-large matrix to $M \times N=2$ dimensions, in order to visualize the individual `word clouds` on a 2D-grid. As type of dimensionality reduction technique a combination between `Principal Component Analysis (PCA)` and subsequent `t-distributed Stochastic Neighbor Embedding (TSNE)` is used.

You can consider the dimensionality reduction approach as `BlackBox`, however, you can also look it up in the respective documentations of `scikit-learn` (will be part of the `Machine Learning Lecture`)
- `TSNE` = https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
- `PCA` = https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA  

In [ ]:
def get_labels_embeddings(word_cloud):
    embeddings=[]
    labels=[]
    for key in word_cloud:
        word_embed = word_cloud.get(key)
        for word in word_embed:
            labels.append(word)
            embeddings.append(word_embed.get(word))
    
    return np.array(labels), np.array(embeddings)

def tsne_plot(word_cloud, topn):
    labels, tokens = get_labels_embeddings(word_cloud)

    if topn != 0:
        colors = cm.rainbow(np.linspace(0, 1, len(labels))) 
    else:
        color = ["orange"]
    
    tsne_model = TSNE(perplexity=4, n_components=2, init='pca', n_iter=3500, random_state=10)
    tsne_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    
    for value in tsne_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 10)) 
    
    for i in range(len(x)):
        if topn != 0:
            if i % (topn+1) == 0:
                color = colors[i-1]
        plt.scatter(x[i],y[i],color=color)
        plt.annotate(labels[i], xy=(x[i], y[i]))
   
    plt.show()

## Build and Train your own Word2Vec-System 

- Train your own `Word2Vec` model using the `gensim Word2Vec` module together with the pre-processed Trump-Tweet sentence corpora `[["my", "first", "sentences"], (...), ["my", "second", "sentence"]]` and the following hyperparameters: `vector_size=300, window=5, min_count=1, workers=4` (have a look at the documentation, what does each parameters mean!)
- Report the vocabulary (`words` and `size`) of your trained `Word2Vec` (Hint: `index_to_key` function)

## Evaluation of the Language Model 
- Evaluate your model by reporting word similarities for `america vs. great`, `great vs. again`, `trump vs. clinton`
- In addition also present the most similar words (top-15) for the word `fake`
- Feel free to try other word combinations with more and different evaluation techniques (as shown above for the pre-trained models)
- You can also visualize your entire vocabulary! Get your vocabulary as a list of words (`index_to_key`), compute the `get_word_embeddings(model, wordlist, topn)` with `topn=0`, because no other similar words should be shown (only target  word). Use the output of `get_word_embeddings(model, wordlist, topn)` and call `tsne_plot` with the same `topn=0`

## Save and Store your final Model
- Store the trained and final `Word2Vec` model 
- Use  the `save` method to store your model at a given output path
- Call the `load` method to load your model based on a given input path. Verfiy the model after loading by computing the $cos(\theta)$ similarity for `trump vs. clinton`